# Aggregate

Applies multiple accumulators sequentially in a single pass over a sequence.

In the example below, `Aggregate` is used to run the following seven
accumulators:

- sum of all numbers
- sum of even numbers
- count of numbers
- smallest number
- largest number
- distinct number of digits across all numbers
- list of numbers

concurrently, and in a single pass, over a sequence of integers:

In [1]:
#r "nuget: morelinq, 4.0.0"

using System.Globalization;
using MoreLinq;
using static System.Console;

Enumerable
    .Range(1, 10)
    .Shuffle()
    .Select(n => new { Num = n, Str = n.ToString(CultureInfo.InvariantCulture) })
    .Aggregate(
        0, (s, e) => s + e.Num,
        0, (s, e) => e.Num % 2 == 0 ? s + e.Num : s,
        0, (s, _) => s + 1,
        (int?)null, (s, e) => s is int n ? Math.Min(n, e.Num) : e.Num,
        (int?)null, (s, e) => s is int n ? Math.Max(n, e.Num) : e.Num,
        new HashSet<int>(), (s, e) => { s.Add(e.Str.Length); return s; },
        new List<(int, string)>(), (s, e) => { s.Add((e.Num, e.Str)); return s; },
        (sum, esum, count, min, max, lengths, items) => new
        {
            Sum           = sum,
            EvenSum       = esum,
            Count         = count,
            Average       = (double)sum / count,
            Min           = min is int mn ? mn : throw new InvalidOperationException(),
            Max           = max is int mx ? mx : throw new InvalidOperationException(),
            UniqueLengths = lengths,
            Items         = items,
        })

Installed Packages morelinq, 4.0.0

{ Sum = 55, EvenSum = 30, Count = 10, Average = 5.5, Min = 1, Max = 10, UniqueLengths = System.Collections.Generic.HashSet`1[System.Int32], Items = System.Collections.Generic.List`1[System.ValueTuple`2[System.Int32,System.String]] } Sum 55 EvenSum 30 Count 10 Average 5.5 Min 1 Max 10 UniqueLengths [ 1, 2 ] Count 2 Comparer System.Collections.Generic.GenericEqualityComparer`1[System.Int32] (values) [ 1, 2 ] Items index value 0 (5, 5) Item1 5 Item2 5 1 (10, 10) Item1 10 Item2 10 2 (2, 2) Item1 2 Item2 2 3 (9, 9) Item1 9 Item2 9 4 (1, 1) Item1 1 Item2 1 5 (4, 4) Item1 4 Item2 4 6 (8, 8) Item1 8 Item2 8 7 (7, 7) Item1 7 Item2 7 8 (3, 3) Item1 3 Item2 3 9 (6, 6) Item1 6 Item2 6

Writing each aggregator this way can be tedious, repetitive and error-prone
because you cannot, for example, reuse [`Enumerable.Sum`][sum] but there is a
set of [_experimental overloads_][exp] (that live in the
[`MoreLinq.Experimental` namespace][expns]) that allow aggregators to be
written as _reactive comprehensions_. This enables you to use aggregators from
[System.Reactive]. The next example is logically the same as the previous,
except it uses the overload where aggregators from System.Reactive are reused.


[sum]: https://docs.microsoft.com/en-us/dotnet/api/system.linq.enumerable.sum
[expns]: https://morelinq.github.io/3.2/ref/api/html/N_MoreLinq_Experimental.htm
[exp]: https://morelinq.github.io/3.2/ref/api/html/Overload_MoreLinq_Experimental_ExperimentalEnumerable_Aggregate.htm
[System.Reactive]: https://www.nuget.org/packages/System.Reactive/

In [2]:
#r "nuget: System.Reactive, 6.0.0"

using System.Reactive.Linq;
using MoreLinq.Experimental;

Enumerable
    .Range(1, 10)
    .Shuffle()
    .Select(n => new { Num = n, Str = n.ToString(CultureInfo.InvariantCulture) })
    .Aggregate(
        s => s.Sum(e => e.Num),
        s => s.Select(e => e.Num).Where(n => n % 2 == 0).Sum(),
        s => s.Count(),
        s => s.Min(e => e.Num),
        s => s.Max(e => e.Num),
        s => s.Select(e => e.Str.Length).Distinct().ToArray(),
        s => s.ToList(),
        (sum, esum, count, min, max, lengths, items) => new
        {
            Sum           = sum,
            EvenSum       = esum,
            Count         = count,
            Average       = (double)sum / count,
            Min           = min,
            Max           = max,
            UniqueLengths = lengths,
            Items         = items,
        })

Installed Packages System.Reactive, 6.0.0

{ Sum = 55, EvenSum = 30, Count = 10, Average = 5.5, Min = 1, Max = 10, UniqueLengths = System.Int32[], Items = System.Collections.Generic.List`1[<>f__AnonymousType0#1`2[System.Int32,System.String]] } Sum 55 EvenSum 30 Count 10 Average 5.5 Min 1 Max 10 UniqueLengths [ 1, 2 ] Items index value 0 { Num = 1, Str = 1 } Num 1 Str 1 1 { Num = 9, Str = 9 } Num 9 Str 9 2 { Num = 4, Str = 4 } Num 4 Str 4 3 { Num = 5, Str = 5 } Num 5 Str 5 4 { Num = 8, Str = 8 } Num 8 Str 8 5 { Num = 7, Str = 7 } Num 7 Str 7 6 { Num = 10, Str = 10 } Num 10 Str 10 7 { Num = 6, Str = 6 } Num 6 Str 6 8 { Num = 2, Str = 2 } Num 2 Str 2 9 { Num = 3, Str = 3 } Num 3 Str 3

For more details, [see the documentation][doc].


[doc]: https://morelinq.github.io/3.2/ref/api/html/Overload_MoreLinq_MoreEnumerable_Aggregate.htm